<a href="https://colab.research.google.com/github/adamlim19/Artifact-Detection/blob/main/Arifact_Detection_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, Conv2D, Flatten, concatenate, MaxPooling2D, AveragePooling2D, Dropout, Activation, Add, Multiply, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sys
import os
import keras.backend as K
from keras.models import load_model
from sklearn.model_selection import train_test_split
#loading dataset
def normalize_img(img):
    img = tf.cast(img, dtype=tf.float32)
    # Map values in the range [-1, 1]
    return (img/tf.squeeze(img).numpy().max())

train = pd.read_csv('/content/gdrive/My Drive/labels_final.csv')
train = pd.DataFrame(train)
print(train)

train_image = []
for i in range(train.shape[0]):
    img = tf.keras.preprocessing.image.load_img('/content/gdrive/My Drive/Images/'+train['Id'][i], color_mode="grayscale",target_size=(512,512))
    img = image.img_to_array(img)
    img = normalize_img(img)
    train_image.append(img)
X = np.array(train_image)

#importing classification model

def Hamming_loss(y_true, y_pred):
    tmp = K.abs(y_true-y_pred)
    return K.mean(K.cast(K.greater(tmp,0.5),dtype=float))

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2*((p*r)/(p+r+K.epsilon()))

y=np.array(train.drop(['Id'], axis=1))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.20, random_state=42)

model = tf.keras.models.load_model('/content/gdrive/My Drive/classification_epoch-89-val_loss-0.00068016-val_acc-0.98222.h5', custom_objects={'Hamming_loss': Hamming_loss, 'recall': recall, 'precision': precision, 'f1': f1})
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Hamming_loss, recall, precision, f1])

predictions_class = model.predict(X_test)

#importing severity model
def q_loss(q,y,f):
    f = tf.cast(f, tf.float32)
    y = tf.cast(y, tf.float32)
    e = (y-f)
    return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)
model = tf.keras.models.load_model('/content/gdrive/My Drive/Regression_epoch-83-val_loss-0.32475540.h5', compile = False)
model.summary()
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer=opt, loss=lambda y,f: q_loss(0.90,y,f))
ypred = model.predict(X_test)

#normalizing predictions
for i in range(len(X_test)):
  ypred[i,0] = ypred[i,0]+0.5
  ypred[i,1] = ypred[i,1]+0.5
  ypred[i,2] = ypred[i,2]+0.5

for i in range(len(X_test)):  
    if ypred[i,0]>1.75:
      ypred[i,0] = ypred[i,0] + 0.75
    if ypred[i,0]>1.5 and ypred[i,0]<1.75:
      ypred[i,0] = ypred[i,0] + 0.5
    if ypred[i,0]<1:
      ypred[i,0] = ypred[i,0]-0.75
      
    if ypred[i,1]>2.25:
      ypred[i,1] = ypred[i,1] + 0.5
    if ypred[i,1]<1.75:
      ypred[i,1] = ypred[i,1]-0.75
      
    if ypred[i,2]>2.25:
      ypred[i,2] = ypred[i,2] + 0.5
    if ypred[i,2]>1.5 and ypred[i,2]<1.75:
      ypred[i,2] = ypred[i,2]-0.5
    if ypred[i,2]<1.5:
      ypred[i,2] = ypred[i,2]-0.25
print(ypred)

#getting output predictions
for j in range(0,len(X_test)):
    score_motion = predictions_class[j][0] * ypred[j][0]
    score_cs = predictions_class[j][1] * ypred[j][1]
    score_rf = predictions_class[j][2] * ypred[j][2]
    print(score_motion, ' ', score_cs, ' ', score_rf)
    plt.imshow(np.squeeze(X_test[j]), cmap="gray")
    plt.show()   
    if score_motion>=2:
      print('Severe Motion')
    if score_motion>=1 and score_motion<2:
      print('Medium Motion')
    if score_motion<1 and score_motion>0.5:
      print('Mild Motion')
    if score_motion<=0.5:
      print('No Motion')
    if score_cs>=2:
      print('Severe cs')
    if score_cs>=1 and score_cs<2:
      print('Medium cs')
    if score_cs<1 and score_cs>0.5:
      print('Mild cs')
    if score_cs<=0.5:
      print('No cs')
    if score_rf>=2:
      print('Severe rf')
    if score_rf>=1 and score_rf<2:
      print('Medium rf')
    if score_rf<1 and score_rf>0.5:   
      print('Mild rf')
    if score_rf<=0.5:
      print('No rf')
    print(' ')